# Introduction
This notebook shows how to create a policy

In [ ]:
%use roboquant(0.9-SNAPSHOT)
Welcome()

## Extending default policy: creating our own order types

We're now ready to develop a "custom policy". For this first example we extend the DefaultPolicy and create our own Order type.

In [ ]:
class MyPolicy(var percentage: Double=0.05) : DefaultPolicy() {

        override fun createOrder(signal: Signal, qty: Double, price: Double): Order? {
            
            // We don't short and  other sell orders are covered by the bracket order.
            if (qty < 0) return null
            
            val asset = signal.asset
      
            return BracketOrder(
                MarketOrder(asset, qty),
                TrailOrder(asset, -qty, percentage/2.0),
                StopOrder(asset, -qty, price* (1 - percentage))
            )
        }
}

## Create a new roboquant and evaluate it
We are now ready to back-test our strategy using the just created Policy. And suppose we want to find out what is the optimal percentage to use in our policy, we'll iterate over 10 values to see what results in the best profit.

In [ ]:
val feed = AvroFeed.sp500()
val strategy = EMACrossover()
val policy = MyPolicy()
val roboquant =  Roboquant(strategy, AccountSummary(), policy = policy)

for (percentage in 1..10) {
    policy.percentage = percentage / 100.0
    roboquant.run(feed, runName = "run-$percentage%")
}

In [ ]:
var data = roboquant.logger.getMetric("account.equity")
MetricChart(data)